In [1]:
# full screen width of Jupyter notebook
from IPython.display import display, HTML
display(HTML("<style>.container {width:80%;} </style>")) 
display(HTML("<style>.toc{ margin-top: 100px;} </style>"))

In [8]:
import os
import sys
os.chdir('../../')
print(f'[+] cwd = {os.getcwd()}, __file__ = {__file__ if "__file__" in vars() else None}')
sys.path.insert(0, os.path.abspath('./'))

import gc
import runningz as rz
from runningz.fe.util.util import get_dm_from_dict, f_fe_cszscore, f_fe_zscore, f_fe_log_abs
from runningz.util.npy import to_mdnpy_and_dump

# ------------------------------------------------------------> input
path_output     = '/Users/runningz/data/mdnpy'
path_dict       = '/Users/runningz/project/runningz/pipeline/cache/00000000/v3/roll_2017_2022.dict'
path_dict_other = None
f_fe_pre        = None # 'cszscore'
args_label      = ['vwap', 1, 5]
label_from_str  = '' # 'ADD__vwap,0,1__vwap,0,2'
dump_labels     = [] # ['vwap,2,3']

# ------------------------------------------------------------> data_config
data_config = rz.load_json(path_dict)['data_config']
data_config['path_input'] = rz.CONST.PATH_DATA
data_config['date_dict'] = {
    'all':     [20221201, 20230201],
    'train':   [20100101, 20221231],
    'valid':   [20230101, 20231231],
}    
data_config['col_label_args'] = args_label
data_config['other_info']['label_from_str'] = label_from_str
data_config['other_info']['col_label_args_list'] = []
if dump_labels is not None:
    for s in dump_labels:
        col_label_args = s.strip().replace(' ', '').split(',')
        col_label_args[1], col_label_args[2] = int(col_label_args[1]), int(col_label_args[2])
        data_config['other_info']['col_label_args_list'].append(col_label_args)
    
# ------------------------------------------------------------> get dm
dm, col_id, col_tid, col_label, cols_tid, cols_not, cols_fea = \
    get_dm_from_dict(path_dict, path_dict_other, None, None, data_config = data_config)
cols_label = [col_label] + [col for col in dm.head().columns if 'LABEL' in col]

# ------------------------------------------------------------> f_fe_xxx
if f_fe_pre is not None:
    for pre in f_fe_pre.split(','): eval(f'f_fe_{pre}')(dm, col_tid, cols_fea)
        
# ------------------------------------------------------------> get dm
rz.TIMER(s = 'dump *.json')
cols_id    = [col_tid, col_id]
df_id      = dm.get_cols_pandas(cols_id)
id_list    = [df_id[col].unique() for col in cols_id]
scale_DICT = dm.scaler.scale_DICT
print(f'[+] id_list = {[len(x) for x in id_list]}, fea = {len(cols_fea)}, label = {cols_label}')
rz.dump_json(cols_fea,   f'{path_output}/cols_fea.json')
rz.dump_json(cols_label, f'{path_output}/cols_label.json')
rz.dump_json(id_list,    f'{path_output}/id_list.json')
# rz.dump_json(scale_DICT, f'{path_output}/scale_DICT.json')

rz.TIMER(s = 'dump data_fea.npy')
dm_dict = {'dm': dm}
to_mdnpy_and_dump(dm_dict, cols_id, cols_fea, [], id_list = id_list, verbose = True,
    path = path_output, batch_size = 200, data_name = 'data_fea.npy')

rz.TIMER(s = 'dump data_label.npy')
to_mdnpy_and_dump(dm_dict, cols_id, cols_label, ['mask'], id_list = id_list, verbose = True, 
    path = path_output, batch_size = 200, data_name = 'data_label.npy')
rz.TIMER(s = 'fe_dump.py SUCC')


[+] cwd = /, __file__ = None
[RZ][22:37:40] XXX/runningz/fe/util/util.py:135-> [+] date_dict.hist = date_dict.train
[RZ][22:37:40] XXX/runningz/fe/util/util.py:137-> [+] data_config
[RZ][22:37:40] XXX/runningz/fe/util/util.py:138-> {
[RZ][22:37:40] XXX/runningz/fe/util/util.py:138->     "data_reader": "BasicData",
[RZ][22:37:40] XXX/runningz/fe/util/util.py:138->     "path_input": "/Users/runningz/workspace/data/tushare/daily",
[RZ][22:37:40] XXX/runningz/fe/util/util.py:138->     "date_dict": {
[RZ][22:37:40] XXX/runningz/fe/util/util.py:138->         "all": [20221201, 20230201],
[RZ][22:37:40] XXX/runningz/fe/util/util.py:138->         "train": [20100101, 20221231],
[RZ][22:37:40] XXX/runningz/fe/util/util.py:138->         "valid": [20230101, 20231231],
[RZ][22:37:40] XXX/runningz/fe/util/util.py:138->         "hist": [20100101, 20221231]
[RZ][22:37:40] XXX/runningz/fe/util/util.py:138->     },
[RZ][22:37:40] XXX/runningz/fe/util/util.py:138->     "cols_dict": {
[RZ][22:37:40] XXX/ru

read_csv_list(): 100%|################################################| 1/1 [00:00<00:00,  6.09it/s]


[RZ][22:37:40] XXX/runningz/fe/prepare_data/base_data.py:215-> [init] <---------- label ----------> 
[RZ][22:37:40] XXX/runningz/fe/prepare_data/base_data.py:248-> [+] label_from_str = , col_label_from = LABEL_vwap_0_1
[RZ][22:37:40] XXX/runningz/fe/prepare_data/base_data.py:249-> [+] col_label_args_list = [['vwap', 0, 1]]
[RZ][22:37:40] XXX/runningz/fe/prepare_data/base_data.py:258-> [+] cols_label = ['LABEL_vwap_0_1', 'LABEL_RAW']
[RZ][22:37:40] XXX/runningz/fe/prepare_data/base_data.py:262-> [+] f_label_pre = cszscore, col_label_from = LABEL_vwap_0_1
[+] total = 1


fe_generator:   0%|          | 0/1 [00:00<?, ?it/s]

[+] total = 1


...

[*] 2024/03/11 22:37:47 -------------------- Timer = 161.40 seconds | dump data_label.npy


  0%|                                                                         | 0/1 [00:00<?, ?it/s]

[2024/03/11 22:37:49] mem_cur = 1.41GB = 1,514,434,560
[2024/03/11 22:37:49] mem_max = 1.41GB = 1,514,434,560


100%|#################################################################| 1/1 [00:01<00:00,  1.76s/it]

[*] 2024/03/11 22:37:49 -------------------- Timer = 163.20 seconds | fe_dump.py SUCC


In [9]:
cols_label

['label', 'LABEL_vwap_0_1', 'LABEL_RAW']